In [2]:
#!pip install kagglehub
import kagglehub
import os
import shutil
from urllib.request import urlretrieve
import numpy as np
import pandas as pd


## Download and unpack data properly

In [3]:
def download_movielens():
    permalink = "https://files.grouplens.org/datasets/movielens/ml-32m.zip"

    # Download the dataset
    urlretrieve(permalink, "movielens.zip")

    # Unzip the dataset
    shutil.unpack_archive("movielens.zip", "data/")

    os.remove("movielens.zip")

def download_imdb():
    path = kagglehub.dataset_download("ashishjangra27/imdb-movies-dataset")
    shutil.move(path, "data/imdb/")

download_movielens()
download_imdb()

KeyboardInterrupt: 

## Load data and merge dataframes

In [4]:
def load_data():
    # Contains movieId, title, genres   
    ml_movies = pd.read_csv('data/ml-32m/movies.csv')
    
    # Contains userId, movieId, rating, timestamp
    ml_ratings = pd.read_csv('data/ml-32m/ratings.csv')
    
    # Contains userId, movieId, tag, timestamp
    ml_tags = pd.read_csv('data/ml-32m/tags.csv')

    # Contains movieId, imdbId, tmdbId
    ml_links = pd.read_csv('data/ml-32m/links.csv')

    # IMDB Dataset:
    imdb = pd.read_csv('data/imdb/movies.csv')

    return ml_movies, ml_ratings, ml_tags, ml_links, imdb

ml_movies, ml_ratings, ml_tags, ml_links, imdb = load_data()

/tmp/ipykernel_1321356/683732697.py:15: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/imdb/movies.csv')


In [5]:
# Preprocessing IMDB:

#drop if description is "Add a plot" because we have no description then
imdb_descriptions = imdb[imdb['description'] != "Add a Plot"]

imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')

#keep what we need
imdb_descriptions = imdb_descriptions[['id', 'description']]

#merge them, so we have a li nk between the two datasets
merged_links = ml_links.merge(imdb_descriptions, left_on='imdbId', right_on='id', how='inner').dropna(subset=['id'])

/tmp/ipykernel_1321356/493108502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = imdb_descriptions['id'].str[2:]
/tmp/ipykernel_1321356/493108502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_descriptions['id'] = pd.to_numeric(imdb_descriptions['id'], errors='coerce')


In [6]:
#merge the movies with the descriptions if that is what we want
ml_movies_description = ml_movies.merge(merged_links, left_on='movieId', right_on='movieId', how='inner')

ml_movies_description
ml_movies_description.to_csv('data/movies_with_description.csv', index=False)